# SETUP

In [1]:
import os
os.getcwd()
os.chdir("..")

In [2]:
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
from datetime import date

In [3]:
from src.features.build_features import mapping_marital_status

In [4]:
pd.options.display.max_rows = 10
pd.options.display.max_columns = 999999

# Data load

In [5]:
df = pd.read_csv("data/raw/marketing_campaign.csv", sep = "\t")
df.columns = df.columns.str.lower()

# Feature engineering

Fixing column types

In [6]:
df['kidhome'] = df['kidhome'].astype('int')
df['teenhome'] = df['teenhome'].astype('int')
df['dt_customer'] = pd.to_datetime(df['dt_customer'], format = "%d-%m-%Y")
df['acceptedcmp1'] = df['acceptedcmp1'].astype('int')
df['acceptedcmp2'] = df['acceptedcmp2'].astype('int')
df['acceptedcmp3'] = df['acceptedcmp3'].astype('int')
df['acceptedcmp4'] = df['acceptedcmp4'].astype('int')
df['acceptedcmp5'] = df['acceptedcmp5'].astype('int')
df['complain'] = df['complain'].astype('int')
df['response'] = df['response'].astype('int')

Columns and rows removal, NA inputation

In [7]:
df = df.drop(columns = ["z_costcontact", "z_revenue"], axis = 1)
df['income'] = df['income'].fillna(df['income'].median())
df = df.loc[~df.marital_status.isin(['Absurd', 'YOLO'])]

Creation of new features

In [8]:
df['age'] = date.today().year - df['year_birth']
df['nb_kids'] = df['kidhome'] + df['teenhome']
df["is_parent"] = np.where(df.nb_kids > 0, 1, 0)
df[['adults', 'marital_status_cleaned']] = df['marital_status'].apply(lambda x: mapping_marital_status(x)).to_list()
df['nb_family_members'] = df['nb_kids'] + df['adults']
df["nb_family_members"] = df["nb_family_members"].astype(int)
df['education_cleaned'] = df['education'].replace({'Basic':'Undergraduate','2n Cycle':'Undergraduate','Graduation':'Postgraduate','Master':'Postgraduate','PhD':'Postgraduate'})
df['marital_status'] = df['marital_status'].replace({'Divorced':'Alone','Single':'Alone','Married':'In couple','Together':'In couple','Absurd':'unknown','Widow':'Alone','YOLO':'unknown'})
df['income_per_person'] = (df['income'] / df['nb_family_members'])
df["income_per_person"] = df["income_per_person"].astype(int)
df['tenure_days'] = pd.to_datetime(date.today(), format = "%Y-%m-%d") - df['dt_customer']
df['tenure_days'] = df['tenure_days'].dt.days
df['total_spendings'] = df['mntwines'] + df['mntfruits'] + df['mntmeatproducts'] + df['mntfishproducts'] + df['mntsweetproducts'] + df['mntgoldprods'] 
df["total_spendings"] = df["total_spendings"].astype(int)
df['wines_share'] = df['mntwines'] / df['total_spendings']
df['fruits_share'] = df['mntfruits'] / df['total_spendings']
df['meat_share'] = df['mntmeatproducts'] / df['total_spendings']
df['fish_share'] = df['mntfishproducts'] / df['total_spendings']
df['sweets_share'] = df['mntsweetproducts'] / df['total_spendings']
df['gold_products_share'] = df['mntgoldprods'] / df['total_spendings']
df['total_spendings_per_person'] = (df['total_spendings'] / df['nb_family_members'])
df["total_spendings_per_person"] = df["total_spendings_per_person"].astype(int)
df["nb_accepted_campaigns"] = df[["acceptedcmp1", "acceptedcmp2", "acceptedcmp3", "acceptedcmp4", "acceptedcmp5"]].gt(0).sum(axis = 1)
df["total_purchases"] = df["numwebpurchases"] + df["numcatalogpurchases"] + df["numstorepurchases"]
df["total_spendings_per_purchase"] = (df['total_spendings'] / df["total_purchases"])
df["total_spendings_per_purchase"] = df["total_spendings_per_purchase"]
df["spendings_to_income"] = df['total_spendings'] / df['income']
df["pct_deals_purchases"] = df["numdealspurchases"] / df["total_purchases"]
df["pct_web_purchasess"] = df["numwebpurchases"] / df["total_purchases"]
df["pct_catalog_purchases"] = df["numcatalogpurchases"] / df["total_purchases"]
df["pct_store_purchases"] = df["numstorepurchases"] / df["total_purchases"]

In [9]:
df["total_spendings_per_purchase"].describe()

count    2236.000000
mean             inf
std              NaN
min         2.666667
25%        13.000000
50%        29.839744
75%        50.015468
max              inf
Name: total_spendings_per_purchase, dtype: float64

Final cleaning

In [10]:
df = df.loc[~((df.total_spendings > 0) & (df.total_purchases == 0)),:]
df = df.set_index('id')

In [11]:
df.head()

,year_birth,education,marital_status,income,kidhome,teenhome,dt_customer,recency,mntwines,mntfruits,mntmeatproducts,mntfishproducts,mntsweetproducts,mntgoldprods,numdealspurchases,numwebpurchases,numcatalogpurchases,numstorepurchases,numwebvisitsmonth,acceptedcmp3,acceptedcmp4,acceptedcmp5,acceptedcmp1,acceptedcmp2,complain,response,age,nb_kids,is_parent,adults,marital_status_cleaned,nb_family_members,education_cleaned,income_per_person,tenure_days,total_spendings,wines_share,fruits_share,meat_share,fish_share,sweets_share,gold_products_share,total_spendings_per_person,nb_accepted_campaigns,total_purchases,total_spendings_per_purchase,spendings_to_income,pct_deals_purchases,pct_web_purchasess,pct_catalog_purchases,pct_store_purchases
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduation,Alone,58138.0,0,0,2012-09-04,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,1,65,0,0,1.0,single,1,Postgraduate,58138,3613,1617,0.392703,0.054422,0.337662,0.106370,0.054422,0.054422,1617,0,22,73.500000,0.027813,0.136364,0.363636,0.454545,0.181818
2174,1954,Graduation,Alone,46344.0,1,1,2014-03-08,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,0,68,2,1,1.0,single,3,Postgraduate,15448,3063,27,0.407407,0.037037,0.222222,0.074074,0.037037,0.222222,9,0,4,6.750000,0.000583,0.500000,0.250000,0.250000,0.500000
4141,1965,Graduation,In couple,71613.0,0,0,2013-08-21,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,0,57,0,0,2.0,in couple,2,Postgraduate,35806,3262,776,0.548969,0.063144,0.163660,0.143041,0.027062,0.054124,388,0,20,38.800000,0.010836,0.050000,0.400000,0.100000,0.500000
6182,1984,Graduation,In couple,26646.0,1,0,2014-02-10,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,0,38,1,1,2.0,in couple,3,Postgraduate,8882,3089,53,0.207547,0.075472,0.377358,0.188679,0.056604,0.094340,17,0,6,8.833333,0.001989,0.333333,0.333333,0.000000,0.666667
5324,1981,PhD,In couple,58293.0,1,0,2014-01-19,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0,41,1,1,2.0,in couple,3,Postgraduate,19431,3111,422,0.409953,0.101896,0.279621,0.109005,0.063981,0.035545,140,0,14,30.142857,0.007239,0.357143,0.357143,0.214286,0.428571


In [12]:
df.to_pickle('data/processed/marketing_campaign_post_feature_engineering_and_cleaning.pickle')

## Profiling report

In [ ]:
pandas_profiling_extended = ProfileReport(df, title = "Report - pandas_profiling - extended")
pandas_profiling_extended.to_notebook_iframe()
pandas_profiling_extended.to_file("reports/Report pandas-profiler - extended data.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]